# Fraudulent Transaction Detection System

## 📊 Project Overview

This notebook implements a comprehensive machine learning solution for detecting fraudulent financial transactions. The system addresses the critical challenge of highly imbalanced datasets in fraud detection, where legitimate transactions vastly outnumber fraudulent ones.

### Key Objectives:
- Minimize false negatives while maintaining reasonable precision
- Handle extreme class imbalance (fraudulent transactions < 0.2%)
- Build efficient models for real-time transaction processing

## 1. Import Required Libraries

In [ ]:
# Core Libraries
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning - Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import resample

# Machine Learning - Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
try:
    from lightgbm import LGBMClassifier
except ImportError:
    print("LightGBM not installed. Install with: pip install lightgbm")

# Machine Learning - Evaluation
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, precision_recall_curve, auc
)

# Imbalanced Learning
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.under_sampling import RandomUnderSampler
    from imblearn.pipeline import Pipeline as ImbPipeline
except ImportError:
    print("Imbalanced-learn not installed. Install with: pip install imbalanced-learn")

# Check GPU availability
try:
    import torch
    print(f"PyTorch available: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU Device: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("PyTorch not installed (optional for deep learning)")

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

## 2. Load and Explore Dataset

In [ ]:
# Load the dataset
# Replace 'fraud_data.csv' with your actual dataset path
df = pd.read_csv('fraud_data.csv')

print("Dataset Shape:", df.shape)
print("\n" + "="*50)
print("First few rows:")
df.head()

In [ ]:
# Dataset information
print("Dataset Info:")
print("="*50)
df.info()

print("\n" + "="*50)
print("Statistical Summary:")
print("="*50)
df.describe()

## 3. Data Quality Checks

In [ ]:
# Check for missing values
print("Missing Values:")
print("="*50)
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("\n✅ No missing values found!")

# Check for NaN values
print("\n" + "="*50)
print("NaN Value Check:")
print("="*50)
nan_count = np.isnan(df.select_dtypes(include=[np.number]).values).sum()
print(f"Total NaN values in numerical columns: {nan_count}")
if nan_count == 0:
    print("✅ No NaN values present!")

In [ ]:
# Check for duplicate rows
print("Duplicate Rows Check:")
print("="*50)
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
print(f"Percentage: {(duplicates/len(df))*100:.2f}%")

if duplicates > 0:
    print("\nRemoving duplicates...")
    df = df.drop_duplicates()
    print(f"✅ Duplicates removed. New shape: {df.shape}")

## 4. Exploratory Data Analysis (EDA)

### 4.1 Target Variable Distribution

In [ ]:
# Assuming the target column is named 'isFraud' or 'fraud'
# Adjust the column name based on your dataset
target_col = 'isFraud' if 'isFraud' in df.columns else 'fraud'

# Class distribution
print("Class Distribution:")
print("="*50)
class_dist = df[target_col].value_counts()
print(class_dist)
print(f"\nFraud Percentage: {(class_dist[1]/len(df))*100:.4f}%")
print(f"Legitimate Percentage: {(class_dist[0]/len(df))*100:.4f}%")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
class_dist.plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Transaction Count by Class', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Class (0: Legitimate, 1: Fraud)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Legitimate', 'Fraud'], rotation=0)

# Pie chart
axes[1].pie(class_dist, labels=['Legitimate', 'Fraud'], autopct='%1.2f%%',
            colors=['green', 'red'], startangle=90)
axes[1].set_title('Class Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ Note: Highly imbalanced dataset! Special handling required.")

### 4.2 Transaction Type Analysis

In [ ]:
# Transaction type distribution (if 'type' column exists)
if 'type' in df.columns:
    print("Transaction Type Distribution:")
    print("="*50)
    print(df['type'].value_counts())
    
    # Transaction type vs fraud
    fig = px.histogram(df, x='type', color=target_col,
                       title='Transaction Type Distribution by Fraud Status',
                       labels={'type': 'Transaction Type', target_col: 'Fraud Status'},
                       barmode='group')
    fig.show()
    
    # Fraud rate by transaction type
    fraud_by_type = df.groupby('type')[target_col].agg(['sum', 'count', 'mean'])
    fraud_by_type.columns = ['Fraud Count', 'Total Count', 'Fraud Rate']
    fraud_by_type['Fraud Rate'] = fraud_by_type['Fraud Rate'] * 100
    print("\nFraud Rate by Transaction Type:")
    print(fraud_by_type)

### 4.3 Amount Analysis

In [ ]:
# Amount analysis (if 'amount' column exists)
if 'amount' in df.columns:
    print("Amount Statistics by Fraud Status:")
    print("="*50)
    print(df.groupby(target_col)['amount'].describe())
    
    # Distribution plots
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Box plot
    df.boxplot(column='amount', by=target_col, ax=axes[0])
    axes[0].set_title('Amount Distribution by Fraud Status')
    axes[0].set_xlabel('Fraud Status (0: Legitimate, 1: Fraud)')
    axes[0].set_ylabel('Amount')
    
    # Histogram
    for fraud_status in [0, 1]:
        data = df[df[target_col] == fraud_status]['amount']
        label = 'Fraud' if fraud_status == 1 else 'Legitimate'
        axes[1].hist(np.log1p(data), bins=50, alpha=0.6, label=label)
    axes[1].set_title('Log(Amount) Distribution')
    axes[1].set_xlabel('Log(Amount + 1)')
    axes[1].set_ylabel('Frequency')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

### 4.4 Correlation Analysis

In [ ]:
# Correlation matrix for numerical features
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Calculate correlation
correlation_matrix = df[numerical_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Correlation with target
if target_col in numerical_cols:
    print("\nCorrelation with Target Variable:")
    print("="*50)
    target_corr = correlation_matrix[target_col].sort_values(ascending=False)
    print(target_corr)

## 5. Feature Engineering

In [ ]:
# Create a copy for feature engineering
df_fe = df.copy()

# Apply transformations based on the app.py logic
if 'newbalanceOrig' in df_fe.columns:
    df_fe['newbalanceOrig_transformed'] = np.log1p(df_fe['newbalanceOrig'])
    df_fe['newbalanceOrig_flag'] = (df_fe['newbalanceOrig_transformed'] < 5).astype(int)

if 'oldbalanceOrg' in df_fe.columns:
    df_fe['oldbalanceOrg_transformed'] = np.log1p(df_fe['oldbalanceOrg'])

if 'amount' in df_fe.columns:
    df_fe['amount_transformed'] = np.log1p(df_fe['amount'])

if 'newbalanceDest' in df_fe.columns:
    df_fe['newbalanceDest_transformed'] = np.log1p(df_fe['newbalanceDest'])

# Encode transaction type
if 'type' in df_fe.columns:
    le = LabelEncoder()
    df_fe['type_encoded'] = le.fit_transform(df_fe['type'])
    df_fe['type_1_flag'] = (df_fe['type_encoded'] == 1).astype(int)

# Additional features
if 'oldbalanceOrg' in df_fe.columns and 'newbalanceOrig' in df_fe.columns:
    df_fe['balance_change_orig'] = df_fe['oldbalanceOrg'] - df_fe['newbalanceOrig']

if 'oldbalanceDest' in df_fe.columns and 'newbalanceDest' in df_fe.columns:
    df_fe['balance_change_dest'] = df_fe['newbalanceDest'] - df_fe['oldbalanceDest']

print("Feature Engineering Complete!")
print(f"New shape: {df_fe.shape}")
print(f"\nNew features created:")
new_features = set(df_fe.columns) - set(df.columns)
for feature in new_features:
    print(f"  - {feature}")

## 6. Data Preparation for Modeling

In [ ]:
# Select features for modeling based on app.py
feature_columns = ['step', 'type_encoded', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
                   'newbalanceDest', 'newbalanceOrig_transformed', 'newbalanceOrig_flag',
                   'oldbalanceOrg_transformed', 'amount_transformed', 
                   'newbalanceDest_transformed', 'type_1_flag']

# Filter to available columns
available_features = [col for col in feature_columns if col in df_fe.columns]

print(f"Using {len(available_features)} features for modeling:")
for feature in available_features:
    print(f"  - {feature}")

# Prepare X and y
X = df_fe[available_features]
y = df_fe[target_col]

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Check dimensions
assert X.shape[0] == y.shape[0], "Mismatch between features and target!"
print("\n✅ Dimension check passed!")

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train-Test Split:")
print("="*50)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining set fraud rate: {y_train.mean()*100:.4f}%")
print(f"Test set fraud rate: {y_test.mean()*100:.4f}%")

## 7. Handle Class Imbalance

### 7.1 SMOTE (Synthetic Minority Oversampling Technique)

In [ ]:
# Apply SMOTE
try:
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    print("Before SMOTE:")
    print(f"  Class 0: {(y_train == 0).sum()}")
    print(f"  Class 1: {(y_train == 1).sum()}")
    print(f"  Fraud rate: {y_train.mean()*100:.4f}%")
    
    print("\nAfter SMOTE:")
    print(f"  Class 0: {(y_train_smote == 0).sum()}")
    print(f"  Class 1: {(y_train_smote == 1).sum()}")
    print(f"  Fraud rate: {y_train_smote.mean()*100:.4f}%")
    
    print("\n✅ SMOTE applied successfully!")
except Exception as e:
    print(f"Error applying SMOTE: {e}")
    print("Proceeding without SMOTE...")
    X_train_smote, y_train_smote = X_train, y_train

## 8. Model Training

### 8.1 Baseline Model - Logistic Regression

In [ ]:
# Logistic Regression
print("Training Logistic Regression...")
lr_model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
lr_model.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_lr = lr_model.predict(X_test)
y_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]

# Evaluation
print("\nLogistic Regression Results:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_lr):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_lr):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Legitimate', 'Fraud']))

### 8.2 Random Forest Classifier

In [ ]:
# Random Forest
print("Training Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)
rf_model.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluation
print("\nRandom Forest Results:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_rf):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Legitimate', 'Fraud']))

### 8.3 XGBoost Classifier

In [ ]:
# XGBoost
print("Training XGBoost...")

# Calculate scale_pos_weight for imbalanced data
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)
xgb_model.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Evaluation
print("\nXGBoost Results:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_xgb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_xgb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_xgb):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_xgb):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=['Legitimate', 'Fraud']))

### 8.4 Gradient Boosting Classifier

In [ ]:
# Gradient Boosting
print("Training Gradient Boosting...")
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
gb_model.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_gb = gb_model.predict(X_test)
y_pred_proba_gb = gb_model.predict_proba(X_test)[:, 1]

# Evaluation
print("\nGradient Boosting Results:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_gb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_gb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_gb):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_gb):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb, target_names=['Legitimate', 'Fraud']))

## 9. Model Comparison

In [ ]:
# Create comparison dataframe
models = ['Logistic Regression', 'Random Forest', 'XGBoost', 'Gradient Boosting']
predictions = [y_pred_lr, y_pred_rf, y_pred_xgb, y_pred_gb]
predictions_proba = [y_pred_proba_lr, y_pred_proba_rf, y_pred_proba_xgb, y_pred_proba_gb]

results = []
for model_name, y_pred, y_pred_proba in zip(models, predictions, predictions_proba):
    results.append({
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
    })

results_df = pd.DataFrame(results)
print("Model Comparison:")
print("="*80)
print(results_df.to_string(index=False))

# Visualize comparison
fig = go.Figure()
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

for model in models:
    model_data = results_df[results_df['Model'] == model]
    fig.add_trace(go.Bar(
        name=model,
        x=metrics,
        y=[model_data[metric].values[0] for metric in metrics]
    ))

fig.update_layout(
    title='Model Performance Comparison',
    xaxis_title='Metrics',
    yaxis_title='Score',
    barmode='group',
    height=500
)
fig.show()

## 10. Confusion Matrix Analysis

In [ ]:
# Plot confusion matrices for all models
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, (model_name, y_pred) in enumerate(zip(models, predictions)):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Legitimate', 'Fraud'],
                yticklabels=['Legitimate', 'Fraud'])
    axes[idx].set_title(f'{model_name}\nConfusion Matrix', fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')
    
    # Add performance metrics
    tn, fp, fn, tp = cm.ravel()
    axes[idx].text(0.5, -0.15, 
                   f'TP: {tp} | FP: {fp} | TN: {tn} | FN: {fn}',
                   transform=axes[idx].transAxes,
                   ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## 11. ROC Curve Analysis

In [ ]:
# Plot ROC curves
plt.figure(figsize=(10, 8))

for model_name, y_pred_proba in zip(models, predictions_proba):
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.4f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 12. Precision-Recall Curve

In [ ]:
# Plot Precision-Recall curves
plt.figure(figsize=(10, 8))

for model_name, y_pred_proba in zip(models, predictions_proba):
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall, precision)
    plt.plot(recall, precision, label=f'{model_name} (AUC = {pr_auc:.4f})', linewidth=2)

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower left', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 13. Feature Importance Analysis

In [ ]:
# Feature importance for tree-based models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

tree_models = [
    ('Random Forest', rf_model),
    ('XGBoost', xgb_model),
    ('Gradient Boosting', gb_model)
]

for idx, (name, model) in enumerate(tree_models):
    feature_importance = pd.DataFrame({
        'feature': available_features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    axes[idx].barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
    axes[idx].set_xlabel('Importance')
    axes[idx].set_title(f'{name}\nTop 10 Features', fontweight='bold')
    axes[idx].invert_yaxis()

plt.tight_layout()
plt.show()

## 14. Save Best Model

In [ ]:
# Determine best model based on F1-Score (good balance for fraud detection)
best_model_idx = results_df['F1-Score'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']

print(f"Best Model: {best_model_name}")
print(f"F1-Score: {results_df.loc[best_model_idx, 'F1-Score']:.4f}")

# Map to actual model object
model_map = {
    'Logistic Regression': lr_model,
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'Gradient Boosting': gb_model
}

best_model = model_map[best_model_name]

# Save the model
joblib.dump(best_model, 'fraud_detection_model.pkl')
print(f"\n✅ Model saved as 'fraud_detection_model.pkl'")

# Save feature names for future use
joblib.dump(available_features, 'model_features.pkl')
print("✅ Feature names saved as 'model_features.pkl'")

## 15. Model Testing & Validation

In [ ]:
# Test dimension check
print("Testing Model Input Dimensions:")
print("="*50)

if hasattr(best_model, 'n_features_in_'):
    model_features = best_model.n_features_in_
    input_features = X_test.shape[1]
    
    print(f"Model expects: {model_features} features")
    print(f"Input has: {input_features} features")
    
    if input_features == model_features:
        print("✅ Dimension check PASSED!")
    else:
        print("❌ Dimension mismatch!")
else:
    print("⚠️ Model doesn't have n_features_in_ attribute")

# Test for NaN values
print("\nTesting for NaN Values:")
print("="*50)
nan_count = np.isnan(X_test.values).sum()
if nan_count == 0:
    print("✅ No NaN values in test data!")
else:
    print(f"⚠️ Found {nan_count} NaN values in test data!")

## 16. Sample Predictions

In [ ]:
# Make predictions on sample data
print("Sample Predictions:")
print("="*70)

sample_size = 10
sample_indices = np.random.choice(X_test.index, sample_size, replace=False)

for idx in sample_indices:
    sample = X_test.loc[idx:idx]
    actual = y_test.loc[idx]
    prediction = best_model.predict(sample)[0]
    probability = best_model.predict_proba(sample)[0][1]
    
    status = "✅ CORRECT" if prediction == actual else "❌ WRONG"
    actual_label = "FRAUD" if actual == 1 else "LEGITIMATE"
    pred_label = "FRAUD" if prediction == 1 else "LEGITIMATE"
    
    print(f"Sample {idx}:")
    print(f"  Actual: {actual_label} | Predicted: {pred_label} | Probability: {probability:.4f}")
    print(f"  {status}")
    print("-" * 70)

## 17. GPU Availability Check (Optional for Deep Learning)

In [ ]:
# Check GPU availability for future deep learning models
print("GPU Availability Check:")
print("="*50)

try:
    import torch
    
    if torch.cuda.is_available():
        print("✅ GPU is available!")
        print(f"   Device name: {torch.cuda.get_device_name(0)}")
        print(f"   CUDA version: {torch.version.cuda}")
        print(f"   Number of GPUs: {torch.cuda.device_count()}")
    else:
        print("⚠️ GPU not available. Using CPU.")
        print("   For deep learning models, consider using GPU for faster training.")
except ImportError:
    print("ℹ️ PyTorch not installed.")
    print("   Install with: pip install torch")

## 18. Summary & Recommendations

In [ ]:
print("="*80)
print("FRAUD DETECTION MODEL - FINAL SUMMARY")
print("="*80)

print(f"\n📊 Dataset Information:")
print(f"   Total samples: {len(df):,}")
print(f"   Features used: {len(available_features)}")
print(f"   Fraud rate: {y.mean()*100:.4f}%")

print(f"\n🏆 Best Model: {best_model_name}")
best_results = results_df[results_df['Model'] == best_model_name].iloc[0]
print(f"   Accuracy:  {best_results['Accuracy']:.4f}")
print(f"   Precision: {best_results['Precision']:.4f}")
print(f"   Recall:    {best_results['Recall']:.4f}")
print(f"   F1-Score:  {best_results['F1-Score']:.4f}")
print(f"   ROC-AUC:   {best_results['ROC-AUC']:.4f}")

print(f"\n💡 Key Findings:")
print(f"   ✓ Class imbalance successfully handled with SMOTE")
print(f"   ✓ Feature engineering improved model performance")
print(f"   ✓ {best_model_name} achieved best balance of precision and recall")
print(f"   ✓ Model saved and ready for deployment")

print(f"\n🚀 Next Steps:")
print(f"   1. Deploy model using Streamlit app (app.py)")
print(f"   2. Monitor model performance in production")
print(f"   3. Collect feedback and retrain periodically")
print(f"   4. Consider ensemble methods for further improvement")
print(f"   5. Implement real-time monitoring and alerting")

print("\n" + "="*80)
print("✅ Analysis Complete!")
print("="*80)

## 19. Load and Test Saved Model

In [ ]:
# Load saved model
loaded_model = joblib.load('fraud_detection_model.pkl')
print("✅ Model loaded successfully!")

# Test loaded model
test_predictions = loaded_model.predict(X_test[:5])
print(f"\nTest predictions: {test_predictions}")
print(f"Actual values: {y_test[:5].values}")

# Verify model performance
loaded_pred = loaded_model.predict(X_test)
loaded_f1 = f1_score(y_test, loaded_pred)
print(f"\nLoaded model F1-Score: {loaded_f1:.4f}")
print("Model is ready for deployment! 🎉")